In [1]:
!pip install -U kaleido --quiet
!pip install --upgrade plotly --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 90.3 MB/s eta 0:00:00


In [2]:
!plotly_get_chrome


Plotly will install a copy of Google Chrome to be used for generating static images of plots.
Chrome will be installed at: None
Do you want to proceed? [y/n] y
Installing Chrome for Plotly...
Chrome installed successfully.
The Chrome executable is now located at: /usr/local/lib/python3.12/dist-packages/choreographer/cli/browser_exe/chrome-linux64/chrome


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import os

print("Libraries imported successfully.")

# Create a directory to save the charts
if not os.path.exists('charts'):
    os.makedirs('charts')
    print("Created 'charts' directory to save visualizations.")

try:
    df = pd.read_excel('Final Lead Data.xlsx')
    print("Dataset loaded successfully from 'Final Lead Data.xlsx'.")
except FileNotFoundError:
    print("Error: 'Final Lead Data.xlsx' not found.")
    print("Please make sure the Excel file is in the correct directory.")
    dummy_data = {
        'ID': range(10),
        'First Name': list('ABCDEFGHIJ'),
        'Email': [f'{l.lower()}@test.com' for l in list('ABCDEFGHIJ')],
        'Gender': ['Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female'],
        'City': ['Mumbai', 'Pune', 'Mumbai', 'Delhi', 'Pune', 'Mumbai', 'Pune', 'Delhi', 'Mumbai', 'Pune'],
        'Colleges': ['College A', 'College B', np.nan, 'College C', 'College B', 'College A', np.nan, 'College C', 'College A', 'College B'],
        'New College Name': [np.nan, np.nan, 'College D', np.nan, np.nan, np.nan, 'College E', np.nan, np.nan, np.nan],
        'Branch/ Specialisation': ['CS', 'IT', 'CS', 'Mech', 'IT', 'CS', 'ENTC', 'Mech', 'CS', 'IT'],
        'Other Branch': [np.nan, np.nan, np.nan, np.nan, np.nan, 'Comp Sci', np.nan, 'Mechanical', np.nan, np.nan],
        'Academic Year': [1.0, 2.0, 3.0, 4.0, 1.0, np.nan, np.nan, np.nan, np.nan, np.nan],
        'What is your current academic year?': [np.nan, np.nan, np.nan, np.nan, np.nan, '1st Year', '2nd Year', '3rd Year', '4th Year', 'Third Year']
    }
    df = pd.DataFrame(dummy_data)
    print("A dummy dataframe has been created to demonstrate the code.")

df_processed = df.copy()

print("\n--- Initial Dataset Information ---")
df_processed.info()

initial_rows = len(df_processed)
df_processed.drop_duplicates(subset='Email', keep='first', inplace=True)
print(f"\n--- Dropped Duplicates ---\nRemoved {initial_rows - len(df_processed)} duplicate entries based on Email. Remaining records: {len(df_processed)}")

df_processed['College'] = df_processed['Colleges'].fillna(df_processed['New College Name'])
df_processed.drop(columns=['Colleges', 'New College Name'], inplace=True)
print("\n--- Consolidated Columns ---\n'Colleges' and 'New College Name' merged into 'College'.")

df_processed['Branch'] = df_processed['Branch/ Specialisation'].fillna(df_processed['Other Branch'])
df_processed.drop(columns=['Branch/ Specialisation', 'Other Branch'], inplace=True)
print("'Branch/ Specialisation' and 'Other Branch' merged into 'Branch'.")

year_map = {
    '1st Year': 1, 'First Year': 1,
    '2nd Year': 2, 'Second Year': 2,
    '3rd Year': 3, 'Third Year': 3,
    '4th Year': 4, 'Fourth Year': 4, 'Final Year': 4
}
df_processed['What is your current academic year?'] = df_processed['What is your current academic year?'].astype(str).map(year_map)

df_processed['Academic_Year'] = df_processed['What is your current academic year?'].fillna(df_processed['Academic Year'])

df_processed.drop(columns=['Academic Year', 'What is your current academic year?'], inplace=True)
print("'Academic Year' and 'What is your current academic year?' merged into 'Academic_Year'.")

df_processed.dropna(subset=['Academic_Year'], inplace=True)
df_processed['Academic_Year'] = df_processed['Academic_Year'].astype(int)

print(f"\n--- Final Cleaned Data ---\nTotal records after cleaning: {len(df_processed)}")
print(df_processed[['First Name', 'College', 'Branch', 'Academic_Year']].head())

current_year = datetime.now().year

def predict_graduation(year):
    years_to_grad = 4 - year
    return current_year + years_to_grad

df_processed['Year_of_Graduation'] = df_processed['Academic_Year'].apply(predict_graduation)

print("\n--- Graduation Year Prediction ---")
print(df_processed[['First Name', 'Academic_Year', 'Year_of_Graduation']].head())

print("\n--- Generating and Saving Interactive Visualizations ---")
print("Note: To save charts as images, you might need to install the 'kaleido' package. Run: pip install kaleido")

gender_counts = df_processed['Gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']
fig_gender = px.pie(gender_counts, names='Gender', values='Count',
                    title='<b>Gender Distribution of Attendees</b>',
                    color_discrete_sequence=px.colors.sequential.Agsunset,
                    hole=0.3)
fig_gender.update_traces(textposition='inside', textinfo='percent+label')
fig_gender.show()
fig_gender.write_image("charts/gender_distribution.png")
print("Saved gender distribution chart.")

top_10_colleges = df_processed['College'].value_counts().nlargest(10).reset_index()
top_10_colleges.columns = ['College', 'Count']
fig_colleges = px.bar(top_10_colleges, x='College', y='Count',
                      title='<b>Top 10 Colleges by Attendee Count</b>',
                      text='Count', color='Count',
                      color_continuous_scale=px.colors.sequential.Viridis)
fig_colleges.update_traces(textposition='outside')
fig_colleges.update_layout(xaxis_tickangle=-45)
fig_colleges.show()
fig_colleges.write_image("charts/top_10_colleges.png")
print("Saved top 10 colleges chart.")


academic_year_counts = df_processed['Academic_Year'].value_counts().reset_index()
academic_year_counts.columns = ['Academic_Year', 'Count']
academic_year_counts['Academic_Year'] = academic_year_counts['Academic_Year'].astype(str) + ' Year'
academic_year_counts = academic_year_counts.sort_values(by='Academic_Year')
fig_academic = px.funnel(academic_year_counts, x='Count', y='Academic_Year',
                         title='<b>Distribution of Students by Academic Year</b>')
fig_academic.show()
fig_academic.write_image("charts/academic_year_distribution.png")
print("Saved academic year distribution chart.")


fig_graduation = px.histogram(df_processed, x='Year_of_Graduation',
                              title='<b>Distribution of Predicted Graduation Year</b>',
                              text_auto=True,
                              color_discrete_sequence=['#4B0082'])
fig_graduation.update_layout(bargap=0.2)
fig_graduation.update_xaxes(type='category')
fig_graduation.show()
fig_graduation.write_image("charts/graduation_year_distribution.png")
print("Saved graduation year distribution chart.")


print("\n--- Task 1 Complete ---")
print("The final dataframe 'df_processed' contains the cleaned data and predicted graduation year.")

df_processed.to_excel('Predicted_Graduation_Data.xlsx', index=False)
print("\nFinal data saved to 'Predicted_Graduation_Data.xlsx'")


Libraries imported successfully.
Created 'charts' directory to save visualizations.
Dataset loaded successfully from 'Final Lead Data.xlsx'.

--- Initial Dataset Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5303 entries, 0 to 5302
Data columns (total 18 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ID                                                      5303 non-null   int64  
 1   First Name                                              5303 non-null   object 
 2   Email                                                   5303 non-null   object 
 3   Gender                                                  200 non-null    object 
 4   City                                                    55 non-null     object 
 5   Created                                                 5303 non-null   object 
 6   Position                      

Saved gender distribution chart.


Saved top 10 colleges chart.


Saved academic year distribution chart.


Saved graduation year distribution chart.

--- Task 1 Complete ---
The final dataframe 'df_processed' contains the cleaned data and predicted graduation year.

Final data saved to 'Predicted_Graduation_Data.xlsx'
